In [2]:
import tensorflow as tf
import numpy as np

In [4]:
class InnerProductLayer(tf.keras.layers.Layer):
    """
    该layer为根据embedding矩阵的输入计算paper中的l_p向量
    """
    def __init__(self,n,field_num,*args,**kwargs):
        """

        :param n: int 指L_p的维度
        :param field_num: int 指特征中有多少个field，例如如果输入有性别类型、年龄段，那么field就是2
        :param args:
        :param kwargs:
        """
        super(InnerProductLayer,self).__init__(*args,**kwargs)
        self.n=n
        self.field_num=field_num

    def build(self, input_shape):
        self.W=tf.Variable(tf.random.truncated_normal(shape=[1,self.n,self.field_num,self.field_num]))

    def call(self, z,**kwargs):
        """

        :param z: tensor shape:[batch_size,field_num,emb_dim]
        :return:
        """
        z=tf.expand_dims(z,axis=1) # [batch_size,1,field_num,emb_dim]
        transpose_z=tf.transpose(z,perm=[0,1,3,2])  # [batch_size,1,emb_dim,field_num]

        P=tf.matmul(z,transpose_z) # [batch_size,1,field_num,field_num] 对每个样本来说，他的嵌入向量矩阵为[field_num,emb_dim]，对其求内积
        L_P=self.W*P # [batch_size,n,field_num,field_num] 然后做线性变换
        L_P=tf.reduce_sum(L_P,axis=[2,3]) # [batch_size,n]
        return L_P

inputs=np.random.random(size=[5,6,3]).astype(np.float32)
inner_layer=InnerProductLayer(4,field_num=6)

inner_layer(inputs)


<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[ 1.4819505 ,  4.0658646 ,  0.9037664 ,  0.2711991 ],
       [ 3.3790822 ,  2.7338052 , -0.6880965 , -1.9817553 ],
       [ 2.947948  ,  3.555347  , -0.9837462 , -0.95452225],
       [ 3.223913  ,  8.262224  , -1.4352503 , -0.7414987 ],
       [ 1.8996357 ,  6.6446342 , -2.805888  , -2.136921  ]],
      dtype=float32)>

In [5]:
class OuterProductLayer(tf.keras.layers.Layer):
    def __init__(self,n,emb_dim,*args,**kwargs):
        super(OuterProductLayer,self).__init__(*args,**kwargs)
        self.n=n
        self.emb_dim=emb_dim

    def build(self, input_shape):
        self.W=tf.Variable(tf.random.truncated_normal(shape=[1,self.n,self.emb_dim,self.emb_dim]))

    def call(self, z, **kwargs):
        """

        :param z: tensor shape:[batch_size,field_num,emb_dim]
        :return:
        """
        # 计算方法就是paper中说的，没啥好谈的
        f_sum=tf.expand_dims(tf.reduce_sum(z,axis=1),axis=2) # [batch_size,emb_dim,1]
        transpose_f_sum=tf.transpose(f_sum,perm=[0,2,1])  # [batch_size,emb_dim,1]
        P=tf.expand_dims(tf.matmul(transpose_f_sum,f_sum),axis=1) # [batch_size,1,emb_dim,emb_dim]
        L_P=self.W*P # [batch_size,n,emb_dim,emb_dim]
        L_P=tf.reduce_sum(L_P,axis=[2,3])
        return L_P

inputs=np.random.random(size=[5,6,3]).astype(np.float32)
outer_layer=OuterProductLayer(4,emb_dim=3)

outer_layer(inputs)

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[-100.68285 ,  163.0145  ,  -41.71023 ,  175.52466 ],
       [ -54.980583,   89.018456,  -22.776995,   95.849976],
       [ -67.933876,  109.99099 ,  -28.143194,  118.43199 ],
       [ -73.81021 ,  119.505295,  -30.577604,  128.67645 ],
       [-100.447876,  162.63405 ,  -41.612896,  175.115   ]],
      dtype=float32)>

In [6]:
class LzLayer(tf.keras.layers.Layer):
    def __init__(self,n,field_num,emb_dim,*args,**kwargs):
        super(LzLayer,self).__init__(*args,**kwargs)
        self.n=n
        self.field_num=field_num
        self.emb_dim=emb_dim

    def build(self, input_shape):
        self.W=tf.Variable(tf.random.truncated_normal(shape=[1,self.n,self.field_num,self.emb_dim]))

    def call(self, z, **kwargs):
        """

        :param z: tensor shape:[batch_size,field_num,emb_dim]
        :param kwargs:
        :return:
        """
        z=tf.expand_dims(z,axis=1) # [batch_size,1,field_num,emb_dim]
        L_Z=z*self.W # [batch_size,n,field_num,emb_dim]
        L_Z=tf.reduce_sum(L_Z,axis=[2,3]) # [batch_size,n]

        return L_Z
inputs=np.random.random(size=[5,6,3]).astype(np.float32)
lz_layer=LzLayer(4,field_num=6,emb_dim=3)

lz_layer(inputs)


<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[ 4.2083025 , -0.8672843 ,  0.22887194,  1.0576658 ],
       [ 3.4540327 , -0.46194398,  1.4278572 , -1.6943581 ],
       [ 1.8267586 , -2.5708835 , -2.0168152 , -0.49889088],
       [ 2.3245084 , -2.7218816 ,  0.12577295, -1.5272183 ],
       [ 1.6866165 , -2.4964826 , -3.2161415 , -0.2379616 ]],
      dtype=float32)>

In [10]:
class ProductNeuralNetworksModel(tf.keras.Model):
    def __init__(self,input_dims,emb_dim,dense_units,d1,product_type="inner",scoring_layer_units=2,*args,**kwargs):
        """

        :param input_dims: [int] 各个field的embedding layer的input_dim，代表某个field的特征有多少个不同的取值
        :param emb_dim: int 嵌入向量的维度
        :param dense_units:
        :param d1: int paper中的l1层的输入维度，指的是paper中的D_1
        :param product_type: str inner or outer
        :param args:
        :param kwargs:
        """
        super(ProductNeuralNetworksModel,self).__init__(*args,**kwargs)
        self.emb_layers=list()
        self.field_num=len(input_dims)
        self.emb_dim=emb_dim
        for input_dim in input_dims:
            self.emb_layers.append(tf.keras.layers.Embedding(input_dim=input_dim,output_dim=emb_dim))
        self.lz_layer=LzLayer(n=d1,field_num=self.field_num,emb_dim=self.emb_dim)
        if product_type=="inner":
            self.product_layer=InnerProductLayer(n=d1,field_num=self.field_num)
        elif product_type=="outer":
            self.product_layer=OuterProductLayer(n=d1,emb_dim=self.emb_dim)
        else:
            raise ValueError()

        self.b1=tf.Variable(tf.zeros(shape=[d1]))
        self.dense_layer=tf.keras.layers.Dense(dense_units)
        self.scoring_layer=tf.keras.layers.Dense(scoring_layer_units,activation=tf.nn.sigmoid)

    def call(self, inputs, training=None, mask=None):
        """

        :param inputs: tensor ,shape :[batch_size, len(self.input_dims)]
        :param training:
        :param mask:
        :return:
        """
        emb_vecs=list()
        for i,emb_layer in enumerate(self.emb_layers):
            emb_vecs.append(emb_layer(inputs[:,i]))

        # emb_vecs里每一个元素都是(batch_size, emb_dim)
        z=tf.stack(emb_vecs,axis=1) # shape :[batch_size, field_num, emb_dim]
        l_z=self.lz_layer(z)
        l_p=self.product_layer(z)

        l1=l_z+l_p+self.b1
        l1=tf.nn.relu(l1)
        l2=self.dense_layer(l1)
        y=self.scoring_layer(l2)
        return y

input_arr=np.random.randint(0,5,size=[5,3])
inner_model=ProductNeuralNetworksModel([5,5,5],emb_dim=2,dense_units=2,d1=3,product_type="inner")
inner_model(input_arr)

<tf.Tensor: shape=(5, 2), dtype=float32, numpy=
array([[0.4918684 , 0.49843407],
       [0.50047046, 0.49994487],
       [0.5006351 , 0.49992555],
       [0.49324763, 0.49785638],
       [0.49482873, 0.4987411 ]], dtype=float32)>